아래는 ALO 기본 설정 및 라이브러리 설치 코드입니다. library 설치 에러가 발생하면 아래 셀을 재실행 하고, 지속적으로 문제가 있을 시 문의바랍니다

In [ ]:
import argparse
import time
import os
os.chdir(os.path.abspath(os.path.join('./alo')))
from src.alo import ALO
alo = ALO(); alo.preset(); pipelines = list(alo.asset_source.keys())
from src.external import external_load_data, external_save_artifacts

In [ ]:
import pandas as pd

def make_ground_truth(path,save_path,data_type='png'):

    path_list = []
    col_names = ['label','image_path']
    
    data_type_dict = {}
    data_type_dict['png'] = ['png']
    data_type_dict['jpg'] = ['jpg']
    data_type_dict['both'] =  ['png','jpg']
    
    
    WALK = os.walk(path)
    i = 0
    for (Root,Dir,fles) in WALK:
        for fle in fles:
            if fle.split('.')[-1] in data_type_dict[data_type]:
                path_list.append([Root.split('/')[-1] , os.path.join(Root,fle)])
                if i<5:
                    print(path_list[-1])
                    i += 1
            
    df = pd.DataFrame(path_list,columns = col_names)
    df.to_csv(save_path,index=False)
    
    return path_list,df


## Ground truth data 만들기 샘플 코드

In [ ]:
# train 혹은 test 데이터 각각에 대해 ground truth 파일을 각각 만들어야 합니다.

# train 혹은 test 데이터가 있는 폴더명을 입력합니다.
dataDir = '/nas001/users/yoonji.suh/sample_data/mnist_png/mnist_png/train/'
# ground truth 파일이 저장될 경로를 확장자 까지 입력합니다.
saveDir = '/home/jovyan/testCsv.csv'

lst,df = make_ground_truth(dataDir,saveDir)
print('complete')
df.head()

## Train workflow

In [ ]:
# 아래는 Train 시 필요한 라이브러리를 설치하는 코드입니다. library 설치 에러가 발생하면 아래 셀을 재실행 해주세요
external_load_data(pipelines[0], alo.external_path, alo.external_path_permission, alo.control['get_external_data'])
pipeline = pipelines[0]
alo.install_steps(pipeline, alo.control["get_asset_source"])

### 1. Input asset 
##### Input asset의 arguments 수정 및 확인
- 필요한경우 input_args의 항목을 ***input_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
# TCR train asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - sampling(2) - train(3))
step = 0 
input_args = alo.user_parameters[pipeline][step]['args'][0].copy()

# 아래 주석을 풀어 input_args를 원하는 값으로 수정합니다. 
# input_args['x_columns'] = ['input_x0','input_x1']
input_args

##### Input asset 실행 

In [ ]:
data = pipe_val = {} # 초기 input asset process 세팅

data_input, config_input = alo.process_asset_step(alo.asset_source[pipeline][step], step, pipeline, data, pipe_val, [input_args])
# data_input: input asset의 결과물입니다. 다음 asset 실행 시 필요합니다. 
# config_input: input asset의 결과 config입니다. 다음 asset실행 시 필요합니다.

# input asset의 결과 dataframe은 data_input['dataframe']으로 확인할 수 있습니다. 
data_input['dataframe'].head(10) 

</br>

### 2. train asset 
##### train asset의 args수정 및 확인
- 필요한경우 IC_args의 항목을 ***IC_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
# IC train asset 순서에 따라 step 순서를 입력합니다. (input(0) - train(1))
step = 1 
ic_args = alo.user_parameters[pipeline][step]['args'][0].copy()

# 아래 주석을 풀어 preprocess_args 수정합니다. 
# train_args['epochs'] = 1
ic_args

##### train asset 실행 

In [ ]:
data_ic, config_ic = alo.process_asset_step(alo.asset_source[pipeline][step], step, pipeline, data_input, config_input.copy(), [ic_args])
# data_train: IC asset의 결과물입니다. 다음 asset 실행 시 필요합니다. 
# config_train: IC asset의 결과 config입니다. 다음 asset실행 시 필요합니다. 

# ic asset의 결과 dataframe은 data_ic['dataframe']으로 확인할 수 있습니다. 
data_ic['dataframe'].head(10) 

In [ ]:
data_ic.keys()

## Inference workflow

In [ ]:
# 아래는 Inference 시 필요한 라이브러리를 설치하는 코드입니다. library 설치 에러가 발생하면 아래 셀을 재실행 해주세요
external_load_data(pipelines[1], alo.external_path, alo.external_path_permission, alo.control['get_external_data'])
pipeline = pipelines[1]
alo.install_steps(pipeline, alo.control["get_asset_source"])

### 1. Input asset 
##### Input asset의 arguments 수정 및 확인
- 필요한경우 input_args의 항목을 ***input_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
# IC inference asset 순서에 따라 step 순서를 입력합니다. (input(0) - inference(1))
step = 0
input_args = alo.user_parameters[pipeline][step]['args'][0].copy()

# 아래 주석을 풀어 input_args를 원하는 값으로 수정합니다. 
# input_args['path_column'] = ['path']
input_args

##### Input asset 실행 

In [ ]:
data = pipe_val = {} # 초기 input asset process 세팅

data_input, config_input = alo.process_asset_step(alo.asset_source[pipeline][step], step, pipeline, data, pipe_val, [input_args])
# data_input: input asset의 결과물입니다. 다음 asset 실행 시 필요합니다. 
# config_input: input asset의 결과 config입니다. 다음 asset실행 시 필요합니다.

# input asset의 결과 dataframe은 data_input['dataframe']으로 확인할 수 있습니다. 
data_input['dataframe'].head(10) 

<br>

### 2. inference asset 
##### inference asset의 args수정 및 확인
- 필요한경우 IC_args의 항목을 ***IC_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
# IC inference asset 순서에 따라 step 순서를 입력합니다. (input(0) - inference(1))
step = 1
ic_args = alo.user_parameters[pipeline][step]['args'][0].copy()

ic_args

##### inference asset 실행 

In [ ]:
data_ic, config_ic = alo.process_asset_step(alo.asset_source[pipeline][step], step, pipeline, data_input, config_input.copy(), [ic_args])
# data_ic: IC asset의 결과물입니다. 
# config_ic: IC asset의 결과 config입니다. 

# ic asset의 결과 dataframe은 data_ic['dataframe']으로 확인할 수 있습니다. 
data_ic['dataframe'].head(10) 